In [1]:
# Importing the neccessary packages for this SQL test
import csv # Used for importing csv files
import pandas as pd # Used for data manipulation
import sqlite3 # Used for scientific and mathematical calculations

In [2]:
# Create Database
# Since we do not have a database, we will use sqlite to create a empty lightweight database and connect to it.

# Connecting to sqlite
conn = sqlite3.connect("sql_test.db")

# Creating a cursor object using the cursor() method. The cursor class is an instance using which we can
# invoke methods that execute SQLite statements, fetch data from the result sets of the queries.
cur = conn.cursor()

In [3]:
# Create Tables
# Let's execute a query that’ll create a 'orders' table with its relevant field names.
# Note that we will have to define their field formats as well.
cur.execute("""
CREATE TABLE IF NOT EXISTS orders (
order_id BIGINT, 
item_id BIGINT,
buyer_id BIGINT,
quantity INT,
order_time DATETIME,
country VARCHAR(50)
)""")


# Next, let's execute a query that’ll create a 'product' table with its relevant field names.
cur.execute("""
CREATE TABLE IF NOT EXISTS product (
brand VARCHAR(50),
product_id BIGINT,
sku_id BIGINT,
price BIGINT
)""")

In [4]:
# Import the orders data using Pandas
# The imported orders data will be stored in the orders table.
# The imported product data will be stored in the product table.

# load the orders data into a Pandas DataFrame
orders = pd.read_csv('Orders.csv')

# load the product data into a Pandas DataFrame
product = pd.read_csv('Product.csv')

# write the data to a sqlite table
orders.to_sql('orders', conn, if_exists='replace', index = False)

# write the data to a sqlite table
product.to_sql('product', conn, if_exists='replace', index = False)

In [5]:
# Fetch orders table results
# Now let's have a look at the orders table by reading the SQL query directly into a Pandas DataFrame.
pd.read_sql('''SELECT * FROM orders LIMIT 5''', conn)

,order_id,item_id,buyer_id,quantity,order_time,country
0,13006128673,11115650053,735072,35,2019/8/31,SG
1,13006128674,11115650054,3556076,24,2019/8/30,SG
2,7797422667,5906944047,686710,28,2019/8/30,SG
3,7797422666,5906944046,753001,25,2019/8/31,SG
4,9801576917,7911098297,899075,16,2019/8/31,SG


In [6]:
# Fetch product table results
# Now let's have a look at the product table by reading the SQL query directly into a Pandas DataFrame.
pd.read_sql('''SELECT * FROM product LIMIT 5''', conn)

,brand,product_id,sku_id,price
0,Apple,2227657594,11115650053,622
1,Apple,2227657594,11115650054,461
2,Apple,4813479739,5906944047,1171
3,Apple,2227657594,11115650056,1011
4,Apple,4813479739,5906944046,521


### Question 1 (10 marks)
How many orders were generated in market ‘MY’ on date 2019/8/30? The expected output will be exactly a number.

Note : order_time is stored as a text in the db and therefore Date format should follow the example here : “2019/8/30”

In [7]:
pd.read_sql('''
SELECT count(*)
FROM orders
WHERE country = 'MY'
AND order_time = '2019/8/30'
''', conn)

,count(*)
0,49


### Question 2 (20 marks)
In e-commerce, GMV is commonly used term, in our writing test, GMV= quantity*price.

Which brand has the biggest GMV on Date 2019/8/31 in market SG? The expected output will be a brand name.

Note: when joining 2 tables DO NOT use table alias due to system constraints. Follow the example below:

TableA JOIN TableB ON TableA.key1 = TableB.key2

In [8]:
# item_id from the orders table and sku_id from the product table are the primary keys.

pd.read_sql('''
SELECT brand
FROM orders
JOIN product
ON orders.item_id = Product.sku_id
WHERE country = 'SG' and order_time = '2019/8/31'
ORDER BY quantity*price LIMIT 1
''', conn)

,brand
0,Huawei


### Question 3 (30 marks)
Please list down these product_ids in descending order from each brand, which have more than 6 skus (6 is not included) and smaller than 10 skus (10 is not included).

In [9]:
pd.read_sql('''
SELECT brand, product_id
FROM product
GROUP BY product_id
HAVING count(sku_id) > 6 and count(sku_id) < 10
''', conn)

,brand,product_id
0,Huawei,68541658
1,Huawei,149024265
2,Huawei,190597238
3,Huawei,240127387
4,Huawei,246768864
5,Huawei,268179164
6,Huawei,308796907
7,Huawei,324746285
8,Huawei,368828981
9,Huawei,390807976


#### Question 4 (40 marks)
Please list products_ids with no sales records in the Order table and their associated Brands.

1) The brand is sorted by alphabetical order

2) The product_ids are listed in ascending order

3) Limit output rows to 20

In [10]:
pd.read_sql('''
SELECT brand, product_id
FROM Product
LEFT JOIN orders
ON orders.item_id = product.sku_id
WHERE Orders.item_id is Null
ORDER BY brand, product_id 
LIMIT 20
''', conn)

,brand,product_id
0,ASUS,2045925408
1,ASUS,2349410789
2,ASUS,2374824788
3,ASUS,2708669306
4,ASUS,4607376301
5,ASUS,7404448460
6,Apple,388533618
7,Apple,485258464
8,Apple,485258464
9,Apple,485258464
